### Project Title : Beer Recommendation

### Introduction
##### Many online businesses rely on customer reviews and ratings. It is especially important in the ecommerce industry where customer engagements are greatly impacted by ratings. Websites rely on rating data to power its recommendation engine to provide the best recommendations that are personalized and most relevant to the user and make profits. 
##### This project focuses on the beer industry/beer online shops. To recommend the users with their potential ideal product based on their past ratings or features of the beers will be the key motivation.

#### Import libraries

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window 
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import math
import os
os.environ["PYSPARK_PYTHON"] = "python3"
import urllib
from pyspark.sql import SparkSession
## Recommendation Engine 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS as ml_als
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

#### Data ETL

In [6]:
## setup spark session
spark = SparkSession \
    .builder \
    .appName("beer review") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
## Create Dataframe and SQL Table
## load data into dataframe and create sql tables
beers = spark.read.load("/FileStore/tables/beer_reviews.csv", format='csv', header = True)
beers.createOrReplaceTempView("beer_reviews")

In [8]:
## Display raw data
display(beers.take(5))

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
10325,Vecchio Birraio,1234817823,1.5,2,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5,47986
10325,Vecchio Birraio,1235915097,3,2.5,3,stcules,English Strong Ale,3,3,Red Moon,6.2,48213
10325,Vecchio Birraio,1235916604,3,2.5,3,stcules,Foreign / Export Stout,3,3,Black Horse Black Beer,6.5,48215
10325,Vecchio Birraio,1234725145,3,3,3.5,stcules,German Pilsener,2.5,3,Sausa Pils,5,47969
1075,Caldera Brewing Company,1293735206,4,4.5,4,johnmichaelsen,American Double / Imperial IPA,4,4.5,Cauldron DIPA,7.7,64883


In [9]:
## The dimension of the dataframe
print((beers.count(), len(beers.columns)))

(1586614, 13)

In [10]:
## Check Null
print("Is there missing value in the dataframe?")
print('beers: {}'.format(beers.count() == beers.na.drop().count()))

Is there missing value in the dataframe?
beers: False

#### In this project, 1.5 millions of beer review data from BeerAdvocate through Kaggle will be analyzed.
#### Reviewer information, beer Information and rating information are provided. 
##### Specifically, 
##### Beer information includes : Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID.
##### Rating infomration includes : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings.
##### Reviewer information includes : Reviewer profilename, Review time.

In [12]:
## Show data types
beers.printSchema()

root
-- brewery_id: string (nullable = true)
-- brewery_name: string (nullable = true)
-- review_time: string (nullable = true)
-- review_overall: string (nullable = true)
-- review_aroma: string (nullable = true)
-- review_appearance: string (nullable = true)
-- review_profilename: string (nullable = true)
-- beer_style: string (nullable = true)
-- review_palate: string (nullable = true)
-- review_taste: string (nullable = true)
-- beer_name: string (nullable = true)
-- beer_abv: string (nullable = true)
-- beer_beerid: string (nullable = true)

In [13]:
## Convert from spark dataframe to pandas dataframe
pandasbeer = beers.toPandas()

In [14]:
## Change reviewer profilename to userid
listOfStr = pandasbeer['review_profilename'].tolist()
my_dict = { i: listOfStr[i] for i in range(0, len(listOfStr) )}
flipped_dict = dict(zip(my_dict.values(), my_dict.keys()))
pandasbeer['review_profilename'] = pandasbeer.review_profilename.map(flipped_dict)

In [15]:
## Convert back to spark dataframe
mybeer = spark.createDataFrame(pandasbeer)

### Exploratory data analysis
#### In the first part of this project, I am going to explore what kind of data we get. 
##### I focused on beer style, since it might be an interesting characteristic that affects people's choice of beer. Other attributes, for example ABV, Style, Brewery should also be thoroughly explored if time allowed.
##### In addition, since we also have Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings information, it might be interesting to see the relationships between beer features and different categories of ratings. It would also be interesting to model the overall ratings using ratings from other categories.

In [17]:
## Size of distinct items
print ("number of distinct users", mybeer.select('review_profilename').distinct().count())
print ("number of distinct beers", mybeer.select('beer_name').distinct().count())
print ("number of distinct beer styles", mybeer.select('beer_style').distinct().count())
print ("number of distinct breweries", mybeer.select('brewery_name').distinct().count())
print ("number of distinct ABV", mybeer.select('beer_abv').distinct().count())

number of distinct users 33388
number of distinct beers 56857
number of distinct beer styles 104
number of distinct breweries 5743
number of distinct ABV 531

#### Explore beer styles

In [19]:
## Unique Beer Styles
unique_beer_style = mybeer.select('beer_style').distinct()
display(unique_beer_style)

beer_style
Baltic Porter
Milk / Sweet Stout
American Dark Wheat Ale
American Amber / Red Lager
Dubbel
Dortmunder / Export Lager
Tripel
Belgian Strong Dark Ale
Belgian Strong Pale Ale
Wheatwine


In [20]:
## The number of type of beers in each beer style
df_style= mybeer \
                    .groupBy('beer_style').count() \
                    .orderBy('count', ascending = False)
display(df_style)

beer_style,count
American IPA,117586
American Double / Imperial IPA,85977
American Pale Ale (APA),63469
Russian Imperial Stout,54129
American Double / Imperial Stout,50705
American Porter,50477
American Amber / Red Ale,45751
Belgian Strong Dark Ale,37743
Fruit / Vegetable Beer,33861
American Strong Ale,31945


In [21]:
##Example of beers of belgian strong dark ale style
display(mybeer.where("beer_style like '%Belgian Strong Dark Ale%'"))

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1075,Caldera Brewing Company,1310518783,4,4,4.5,1585716,Belgian Strong Dark Ale,4,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1326071409,3.5,3.5,4,1581037,Belgian Strong Dark Ale,2,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1321407186,3,4,2,1584275,Belgian Strong Dark Ale,3,2.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1311736668,4,4,4,1584282,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1321070530,3.5,3.5,3.5,1584289,Belgian Strong Dark Ale,3,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1320555535,4,3.5,4,1574796,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1319921102,3.5,3.5,4,1582325,Belgian Strong Dark Ale,3.5,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1319858005,3.5,3.5,4,1577029,Belgian Strong Dark Ale,4,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1317515953,4,3,4,1585696,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1317093500,3,3.5,4,1579574,Belgian Strong Dark Ale,3.5,3.5,Vas Deferens Ale,8.1,58920


#### Explore the overall ratings of beers

In [23]:
## The number of ratings of each scores
df_rate= mybeer \
                    .groupBy('review_overall').count() \
                    .orderBy('review_overall', ascending = False)
display(df_rate)

review_overall,count
5,91320
4.5,324385
4,582764
3.5,301817
3,165644
2.5,58523
2,38225
1.5,12975
1,10954
0,7


#### The average rating of beers from each beer style

In [25]:
pandasbeer['review_overall'] = pandasbeer['review_overall'].apply(pd.to_numeric)
pandasbeer.groupby("beer_style")["review_overall"].mean().sort_values(ascending = False).head(10)

Out[17]: beer_style
American Wild Ale 4.093262
Gueuze 4.086287
Quadrupel (Quad) 4.071630
Lambic - Unblended 4.048923
American Double / Imperial Stout 4.029820
Russian Imperial Stout 4.023084
Weizenbock 4.007969
American Double / Imperial IPA 3.998017
Flanders Red Ale 3.992722
Rye Beer 3.981737
Name: review_overall, dtype: float64

#### The beers that have the most ratings

In [27]:
beerrated = pandasbeer.groupby('beer_name')['review_overall'].count().sort_values(ascending=False).head(5)

#### The number of ratings that each reviewer gave

In [29]:
pandasbeer.groupby('review_profilename')['review_overall'].count().sort_values(ascending=False).head(5) 

Out[19]: review_profilename
1586535 5817
1586244 4661
1586246 4617
1584562 3518
1586447 3497
Name: review_overall, dtype: int64

##### Although using this interesting dataset from BeerAdvocate, the project can go into many directions.
##### In the rest part of this project, I am going to dive directly into predicting beer ratings only from review ID, and ratings that they gave to other beers.

### Start to predict beer ratings using recommendation algorithms with Spark MLlib APIs

In [32]:
## Create a new dataframe only focusing on the beer, reviewer and the ratings that give.
df_rating_data = mybeer.select("review_profilename","beer_beerid","review_overall")
df_rating_data = df_rating_data \
            .withColumn("review_profilename", df_rating_data.review_profilename.cast(IntegerType())) \
            .withColumn("beer_beerid", df_rating_data.beer_beerid.cast(IntegerType())) \
            .withColumn("review_overall", df_rating_data.review_overall.cast(DoubleType())) 

In [33]:
## Train and test split
data, hold_out = df_rating_data.randomSplit([0.8, 0.2], seed = 7856)
data.cache()
hold_out.cache()

Out[21]: DataFrame[review_profilename: int, beer_beerid: int, review_overall: double]

#### Use ALS (Alternating Least Square) and collaborative filtering to predict the ratings for the movies
#### ALS machine learning model referred from 
##### https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [35]:
# Build the recommendation model using ALS on the training data

# Specify model with parameters
als = ml_als(userCol="review_profilename", itemCol="beer_beerid", ratingCol="review_overall", coldStartStrategy="drop")

# Use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder()\
    .addGrid(als.maxIter,[5]) \
    .addGrid(als.rank, [4, 6, 8])\
    .addGrid(als.regParam, [0.6, 0.8, 1])\
    .build()

# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_overall",
                                predictionCol="prediction")

tvs = CrossValidator(estimator=als,
                     estimatorParamMaps=paramGrid,
                     evaluator=evaluator,
                     numFolds = 5)

# fit data
myalsmodel = tvs.fit(data)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

#### Evaluate the model by computing the RMSE on the test data

In [37]:
# Get the best model from cross validation, evaluate the best model on test data
best_model = myalsmodel.bestModel
predictions = best_model.transform(hold_out)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.8359151019997325

In [38]:
#Print evaluation metrics and model parameters
print ("RMSE = "+str(rmse))
print ("**Best Model**")
print (" Rank:"+str(best_model._java_obj.parent().getRank())), 
print (" MaxIter:"+str(best_model._java_obj.parent().getMaxIter())), 
print (" RegParam:"+str(best_model._java_obj.parent().getRegParam()))

RMSE = 0.8359151019997325
**Best Model**
 Rank:8
 MaxIter:5
 RegParam:0.6

#### Show predicted ratings

In [40]:
## Show predictions
predictions.show()

+------------------+-----------+--------------+----------+
review_profilename|beer_beerid|review_overall|prediction|
+------------------+-----------+--------------+----------+
 1583334| 148| 3.5| 3.418921|
 1571478| 148| 3.5| 3.319425|
 1582519| 148| 4.5| 3.7294226|
 1561680| 148| 3.0| 3.6078615|
 1572700| 148| 3.5| 3.524527|
 1582440| 148| 4.0| 3.4189177|
 1583748| 148| 4.0| 3.2606394|
 1585322| 148| 3.0| 2.9531577|
 1586453| 148| 4.0| 3.2597518|
 1526994| 148| 3.0| 3.3625593|
 1578820| 148| 4.0| 3.4886796|
 1583094| 148| 4.5| 3.4198446|
 1586160| 148| 3.5| 3.1655352|
 1572053| 148| 4.0| 3.3900034|
 1585640| 148| 4.0| 3.5495143|
 1586233| 148| 4.0| 3.3505538|
 1577354| 148| 5.0| 3.6470146|
 1586290| 148| 4.0| 3.4591606|
 1554931| 148| 4.0| 3.145416|
 1583402| 148| 4.0| 3.3896637|
+------------------+-----------+--------------+----------+
only showing top 20 rows

#### Generate the top 5 beer recommendations for each user
#### Generate the top 5 user recommendations for each beer

In [42]:
# Generate top 5 beer recommendations for user whose id is 666
userRecs = myalsmodel.bestModel.recommendForAllUsers(5).cache()
display(userRecs)

review_profilename,recommendations
6466,"List(List(66833, 4.897217), List(57390, 4.746152), List(54487, 4.746152), List(52103, 4.7447324), List(25156, 4.721709))"
16503,"List(List(66833, 5.023193), List(57390, 4.863244), List(54487, 4.863244), List(52103, 4.863231), List(25156, 4.836565))"
22373,"List(List(66833, 5.5769124), List(52103, 5.3992586), List(57390, 5.399226), List(54487, 5.399226), List(25156, 5.370142))"
29814,"List(List(66833, 4.7190776), List(52103, 4.5347376), List(57390, 4.5218163), List(54487, 4.5218163), List(32842, 4.4969707))"
83242,"List(List(66833, 6.1189184), List(52103, 5.9228454), List(57390, 5.92234), List(54487, 5.92234), List(25156, 5.89007))"
83250,"List(List(66833, 4.895135), List(52103, 4.7382765), List(57390, 4.737872), List(54487, 4.737872), List(25156, 4.712056))"
111515,"List(List(66833, 6.44459), List(25156, 6.2547884), List(25919, 6.2547884), List(54487, 6.2541785), List(57390, 6.2541785))"
227038,"List(List(66833, 5.846277), List(54487, 5.662239), List(57390, 5.662239), List(52103, 5.6617), List(25919, 5.634468))"
234425,"List(List(66833, 6.1762123), List(57390, 5.987368), List(54487, 5.987368), List(52103, 5.985317), List(25156, 5.9609036))"
251316,"List(List(66833, 5.229596), List(57390, 5.0710864), List(54487, 5.0710864), List(52103, 5.069012), List(25156, 5.0524073))"


In [43]:
# Generate top 5 user recommendations for each beer
beerRecs = myalsmodel.bestModel.recommendForAllItems(5).cache()
display(beerRecs)

beer_beerid,recommendations
148,"List(List(1308453, 6.2361217), List(1297167, 6.149966), List(1306753, 6.1194077), List(1306756, 6.1194077), List(1306868, 6.1194077))"
463,"List(List(1308453, 4.5561676), List(1297167, 4.4927406), List(1306753, 4.471211), List(1306756, 4.471211), List(1306868, 4.471211))"
496,"List(List(1308453, 6.9697437), List(1297167, 6.876135), List(1306753, 6.84391), List(1306756, 6.84391), List(1306868, 6.84391))"
833,"List(List(1308453, 5.6635475), List(1297167, 5.5843306), List(1306753, 5.5588803), List(1306756, 5.5588803), List(1306868, 5.5588803))"
1088,"List(List(1308453, 6.1214824), List(1297167, 6.0383267), List(1306753, 6.0105295), List(1306756, 6.0105295), List(1306868, 6.0105295))"
1238,"List(List(1308453, 5.8580604), List(1297167, 5.7759666), List(1306753, 5.747763), List(1306756, 5.747763), List(1306868, 5.747763))"
1580,"List(List(1308453, 5.1874514), List(1297167, 5.116663), List(1306753, 5.0925035), List(1306756, 5.0925035), List(1306868, 5.0925035))"
1591,"List(List(1308453, 5.416042), List(1297167, 5.3411527), List(1306753, 5.3164186), List(1306756, 5.3164186), List(1306868, 5.3164186))"
1645,"List(List(1308453, 5.566258), List(1297167, 5.488978), List(1306753, 5.4619555), List(1306756, 5.4619555), List(1306868, 5.4619555))"
1959,"List(List(1308453, 5.743811), List(1297167, 5.6639447), List(1306753, 5.6372523), List(1306756, 5.6372523), List(1306868, 5.6372523))"


### Discussion
#### Matrix factorization with ALS algorithm was used in the rating prediction task.
#### I used five-fold cross validation to train the model, selected the best model and evaluated on the testing data. 
#### Parameters including max iteration, regularization parameter, and rank were selected using grid search method. 
#### The best model showing the rank number gives us an insight of the number of the latent features in the ALS algorithm. The feature might be related to beer style, abv, brewery, aroma, appearance, palate, tasting and etc. A thorough exploration could be done latter. The findings might help in reinforcing the model using other method to improve the model in the future.
#### The predicted beer ratings and recommendations for customers are also shown above, providing recommended beers to the specific customers.
#### More explorations could be done using this dataset in possible future projects.
##### I focused on beer style, since it might be an interesting characteristic that affects people's choice of beer. Other attributes, for example ABV, Style, Brewery should also be thoroughly explored if time allowed.
##### In addition, since we also have Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings information, it might be interesting to see the relationships between beer features and different categories of ratings. It would also be interesting to model the overall ratings using ratings from other categories, at least plot the distribution of theses ratings or run a simple linear regression.

### Appendix for all the columns in the dataset 
##### Reviewer Information : Reviewer profilename, Review time
###### review_profilename 
###### review_time
##### Rating Information : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings
###### review_overall
###### review_aroma
###### review_appearance
###### review_palate
###### review_taste
##### Beer Information: Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID
###### beer_name
###### beer_abv
###### beer_beerid
###### beer_style
###### brewery_name
###### brewery_id

### References 
##### https://www.kaggle.com/rdoume/beerreviews
##### https://hub.packtpub.com/building-recommendation-engine-spark/ 
##### https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/ 
##### https://blog.statsbot.co/recommendation-system-algorithms-ba67f39ac9a3